In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt
from cmocean import cm
from cartopy.crs import LambertAzimuthalEqualArea
from matplotlib.tri import Triangulation
from scipy.ndimage import gaussian_filter
from scipy.interpolate import RectBivariateSpline
from scipy.spatial import KDTree

import sys
sys.path.insert(0, '../')
from utils import remesh, optimize_mesh, compute_mapping, get_area_ratio
from utils import *
from remeshing import *
import ipdb


%matplotlib inline

In [ ]:
ifile = '../mesh_arctic_ease_25km_max7.npz'
srs_dst = LambertAzimuthalEqualArea(central_longitude=0, central_latitude=90)

In [ ]:
with np.load(ifile) as ds:
    x0 = ds['x']
    y0 = ds['y']
    t0 = ds['t']
    xc = ds['xc']
    yc = ds['yc']
    mask = ds['mask']

In [ ]:
#ex = x0[t0].mean(axis=1)
#ey = y0[t0].mean(axis=1)
#tri0 = Triangulation(x0, y0, t0)
#t0 = t0[(ex > -600) * (ex < 600) * (ey > -600) * (ey < 600)]
#tri0 = clean_mesh(Triangulation(x0, y0, t0))
#tri0 = remove_hanging_elements(tri0)
#plt.triplot(tri0, 'k-', lw=0.1)

In [ ]:
force = False
min_edge_length = 13
max_edge_length = 50
min_edge_angle = 15
min_triangles = 5
search_dist = 50
max_dist0 = 50
min_area = 20
max_mask_fix_node=0.15
max_mask_zero_speed=0.6

m = gaussian_filter(mask.astype(float), 1, truncate=1)
mask = RectBivariateSpline(xc, yc, m[::-1], kx=1, ky=1)(y0, x0, grid=False)
fixed_nodes_idx = np.nonzero(mask < max_mask_fix_node)[0]
#zero_speed = None#np.nonzero(mask < max_mask_zero_speed)[0]

c0 = x0[t0].mean(axis=1)**2

In [ ]:
def get_min_area_for_valid_elemenst(x, y, t, area, fixed_nodes_idx=None):
    if fixed_nodes_idx is None:
        min_area_i = np.argmin(area)
        return area[min_area_i], min_area_i
    fixed_x_flag = np.zeros(x.size, dtype=bool)
    try:
        fixed_x_flag[fixed_nodes_idx] = True
    except:
        ipdb.set_trace()
    fixed_elem_flag = fixed_x_flag[t].sum(axis=1) == 3
    area_min = area[~fixed_elem_flag].min()
    min_area_i = np.nonzero(area == area_min)[0][0]
    return area_min, min_area_i

def remove_element(x, y, t, fixed_nodes_idx=None, neg_tri_i=None):
    if fixed_nodes_idx is None:
        fixed_nodes_idx = []
    neg_tri = t[neg_tri_i]
    neg_tri_i_edges = np.array([[neg_tri[i], neg_tri[(i+1)%3].item()] for i in range(3)])
    edge_lengths = np.hypot(x[neg_tri_i_edges[:,0]] - x[neg_tri_i_edges[:,1]], y[neg_tri_i_edges[:,0]] - y[neg_tri_i_edges[:,1]])
    valid_edges = np.nonzero([(edge[0] not in fixed_nodes_idx) or (edge[1] not in fixed_nodes_idx) for edge in neg_tri_i_edges])[0]
    min_valid_edge = neg_tri_i_edges[valid_edges[np.argmin(edge_lengths[valid_edges])]]
    if min_valid_edge[0] in fixed_nodes_idx:
        good_node = min_valid_edge[0].item()
        bad_node = min_valid_edge[1].item()
    else:
        good_node = min_valid_edge[1].item()
        bad_node = min_valid_edge[0].item()

    new_triangles = np.array(t)
    new_triangles[new_triangles == bad_node] = good_node
    new_triangles = new_triangles[np.sum(new_triangles == good_node, axis=1) < 2]
    new_x = np.array(x)
    new_y = np.array(y)
    new_x[bad_node] = 0
    new_y[bad_node] = 0
    return new_x, new_y, new_triangles

def replace_negative_element_with_two(x, y, t, neg_area_i):
    neg_tri_x = x[t[neg_area_i]].flatten()
    neg_tri_y = y[t[neg_area_i]].flatten()
    potential_trouble_points = np.column_stack([neg_tri_x, neg_tri_y])
    tri_i = find_triangles_for_points(x, y, t, potential_trouble_points)
    contain_tri_i = tri_i[tri_i > 0][0]

    trouble_node = t[neg_area_i][tri_i > 0][0]
    flipped_tri = t[neg_area_i]
    contain_tri = t[contain_tri_i]
    common_edge = np.intersect1d(flipped_tri, contain_tri)
    opposite_node = list(set(list(contain_tri)) - set(list(flipped_tri)))[0]
    new_tri1 = np.array([common_edge[0], opposite_node, trouble_node])
    try:
        new_tri2 = np.array([common_edge[1], opposite_node, trouble_node])
    except:
        ipdb.set_trace()
    new_tri1 = new_tri1[order_cclockwise(x[new_tri1], y[new_tri1])]
    new_tri2 = new_tri2[order_cclockwise(x[new_tri2], y[new_tri2])]
    new_triangles = np.array(t)
    good_new_tri = np.ones(new_triangles.shape[0], dtype=bool)
    good_new_tri[neg_area_i] = False
    good_new_tri[contain_tri_i] = False
    new_triangles = new_triangles[good_new_tri]
    new_triangles = np.vstack([new_triangles, new_tri1, new_tri2])
    return new_triangles

def remove_small_elements(tri, fixed_nodes_idx, min_area=20, max_iter=100):
    x, y, t = tri.x, tri.y, tri.triangles
    area = get_area(x, y, t)
    area_min, min_area_i = get_min_area_for_valid_elemenst(x, y, t, area, fixed_nodes_idx)

    n = 0
    while area_min < min_area:
        #print(n, area_min)
        x, y, t = remove_element(x, y, t, fixed_nodes_idx, min_area_i)
        area = get_area(x, y, t)

        if area.min() < 0 and area.min() < area_min:
            #print('Replace with two')
            t = replace_negative_element_with_two(x, y, t, np.argmin(area))
            area = get_area(x, y, t)
        area_min, min_area_i = get_min_area_for_valid_elemenst(x, y, t, area, fixed_nodes_idx)

        n += 1
        if n >= max_iter:
            break
    return Triangulation(x, y, t), n

def optimize_mesh(tri_a, tri_r, fixed_nodes_idx=None, min_ap_ratio=0.18, min_area=20, verbose=False):
    _, same_in_r = get_same_elements(tri_a.triangles, tri_r.triangles)
    new_elems = np.ones(tri_r.triangles.shape[0], dtype=bool)
    new_elems[same_in_r] = False
    _, _, _, ap_r = measure(tri_r.x, tri_r.y, tri_r.triangles)
    new_elems[ap_r < min_ap_ratio] = True
    # default:
    #new_elems[tri_r.neighbors[new_elems].flatten()] = True
    #new_elems[tri_r.neighbors[new_elems].flatten()] = True
    # 1:  less elements to optimize, only one layer of neighbours
    #new_elems[tri_r.neighbors[new_elems].flatten()] = True
    # 5: no neighbours


    if fixed_nodes_idx is None:
        fixed_nodes = None
    else:
        fixed_nodes = set(list(np.unique(tri_r.triangles))) - set(list(np.unique(tri_r.triangles[new_elems])))
        fixed_nodes = fixed_nodes.union(set(list(fixed_nodes_idx)))
        fixed_nodes = np.array(list(fixed_nodes))

    # default
    #tri = laplace_smooth(tri_r, iterations=40, factor=0.5, fixed_nodes=fixed_nodes)
    # 2:
    tri = laplace_smooth(tri_r, iterations=20, factor=0.5, fixed_nodes=fixed_nodes)

    tri, n = remove_small_elements(tri, fixed_nodes_idx, min_area=min_area)
    if verbose: print(f'Removed {n} small elements')
    tri = clean_mesh(tri)
    return tri

def remesh(tri_a, fixed_nodes_idx, min_area, min_edge_length, min_edge_angle, max_edge_length, verbose=False):
    tri, n = remove_small_elements(tri_a, fixed_nodes_idx, min_area)
    if verbose: print(f'Removed {n} small elements')

    tri, n = collapse_short_edges(tri, min_edge_length, fixed_nodes_idx=fixed_nodes_idx)
    tri = remove_hanging_elements(tri)
    if verbose: print(f'Collapsed {n} short edges')

    tri, n = collapse_small_angle_edges(tri, min_edge_angle, fixed_nodes_idx=fixed_nodes_idx)
    tri = remove_hanging_elements(tri)
    if verbose: print(f'Collapsed {n} small angle edges')

    tri, n = remove_long_edges(tri, max_edge_length, fixed_nodes_idx=fixed_nodes_idx, maxiter=100)
    if verbose: print(f'Removed {n} long edges')
    return tri


In [ ]:
for i in range(200):
    u = -y0 / 20
    v = x0 / 20
    dist_to_center = np.hypot(x0, y0)

    # 0
    u[dist_to_center > 300] = 0
    v[dist_to_center > 300] = 0

    # 3
    u[dist_to_center > 300] = 0
    v[dist_to_center > 300] = 0

    mask = RectBivariateSpline(xc, yc, m[::-1], kx=1, ky=1)(y0, x0, grid=False)
    fixed_nodes_idx = np.nonzero(mask < max_mask_fix_node)[0]

    fig, ax = plt.subplots(1,1, figsize=(5, 5))
    # 0
    #ax.tripcolor(x0, y0, t0, c0, cmap='jet', clim=[0, 80000], edgecolors='k', linewidth=0.1)
    # 4
    ax.tripcolor(x0, y0, t0, c0, cmap='jet', clim=[0, 60000], edgecolors='k', linewidth=0.1)
    ax.quiver(x0, y0, u, v, scale=1, scale_units='xy', angles='xy')
    ax.set_xlim([-500, 500])
    ax.set_ylim([-500, 500])
    ax.set_aspect('equal')
    plt.savefig(f'advect_{i:03d}.png', bbox_inches='tight', pad_inches=0.1)
    plt.close()

    tri0 = Triangulation(x0, y0, t0)
    x1, y1, t1 = x0 + u, y0 + v, t0
    tri_a = Triangulation(x1, y1, t0)
    tri_r = remesh(tri_a, fixed_nodes_idx, min_area, min_edge_length, min_edge_angle, max_edge_length, verbose=True)
    tri_o = optimize_mesh(tri_a, tri_r, fixed_nodes_idx)
    src2dst, weights = compute_mapping(tri_a, tri_o, search_dist)
    area_ratio = get_area_ratio(tri0, tri_a, tri_o, src2dst, weights)
    c0adv = np.zeros(tri_o.triangles.shape[0]) + 0
    np.add.at(c0adv, src2dst[:,1], c0[src2dst[:,0]] * weights)

    x0, y0, t0, c0 = tri_o.x, tri_o.y, tri_o.triangles, c0adv


In [ ]:
#ffmpeg -y -r 10 -f image2 -i advect_%03d.png -vf "scale=trunc(iw/2)*2:trunc(ih/2)*2" -vcodec libx264 -crf 1 -pix_fmt yuv420p advect_disk_5.mp4
#ffmpeg -y -r 10 -f image2 -i frame_%04d_zoom.png -vf "scale=trunc(iw/2)*2:trunc(ih/2)*2" -vcodec libx264 -crf 1 -pix_fmt yuv420p advect_myi_zoom.mp4